In [1]:
import requests
import urllib3
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import os.path
%matplotlib inline

In [2]:
company_listing = "http://www.nepalstock.com.np/company?_limit=500"

In [3]:
http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',company_listing)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
company=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
col = 0
notfirstrun = False
for row in rows:
    companydata =[]
    for data in row:
        if col == 5 and notfirstrun:
            companydata.append(data.a.get('href').split('/')[-1])
        else:
            companydata.append(data.text.strip())
        col += 1
    company.append(companydata)
    col =0
    notfirstrun = True

df = pd.DataFrame(company[1:],columns=company[0])
df.rename(columns={'Operations':'Symbol No'},inplace=True)
df.index.name = "SN"
df.drop(columns='',inplace=True)
df.drop(columns='S.N.',inplace=True)
df.to_json('CompanyList.json',orient='index')
print('There are %s Companies'%len(df.index))
df.head()

There are 209 Companies


,Stock Name,Stock Symbol,Sector,Symbol No
SN,,,,
0,Agriculture Development Bank Limited,ADBL,Commercial Banks,397
1,Api Power Company Ltd.,API,Hydro Power,697
2,Arambha Microfinance Bittiya Sanstha Ltd.,AMFI,Microfinance,2777
3,Arun Kabeli Power Ltd.,AKPL,Hydro Power,2757
4,Arun Valley Hydropower Development Co. Ltd.,AHPC,Hydro Power,360


In [ ]:
# Getting Company Details
symbol = "NMB"
url = "http://www.nepalstock.com/company/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
table = soup.find("table")
print ("Company: ",table.findAll("td")[0].string)
for row in table.findAll("tr")[4:]:
    col = row.findAll("td")
    print (col[0].string,": ",col[1].string)

# Daily FloorSheet Data

In [ ]:
DailyFloorSheet="http://www.nepalstock.com.np/main/floorsheet/index/0/?_limit=5000"

http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',DailyFloorSheet)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
FloorSheet=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
for row in rows:
    FloorSheet.append([data.text.strip() for data in row])
FloorSheetdf = pd.DataFrame(FloorSheet[1:-1],columns=FloorSheet[0])
FloorSheetdf.head()

In [ ]:
df['date'] = pd.to_datetime(FloorSheetdf['Contract No'],format='%Y%m%d%H%M%f').dt.date
df['date']

# CompanyStocks Transactions

In [5]:
def CompanyStocksTransactions(SymbolNo,startDate):
    url="http://www.nepalstock.com.np/company/transactions/%s/0/?startDate=%s&endDate=&_limit=9000000"%(SymbolNo,startDate)
    print("Connecting to %s "%url)
    http = urllib3.PoolManager()
    http.addheaders = [('User-agent', 'Mozilla/61.0')]
    web_page = http.request('GET',url)
    print("Adding to DataFrame")
    soup = BS(web_page.data, 'html5lib')
    table = soup.find('table')
    FloorSheet=[]
    rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
    for row in rows:
        FloorSheet.append([data.text.strip() for data in row])
    FloorSheetdf = pd.DataFrame(FloorSheet[1:],columns=FloorSheet[0])
    FloorSheetdf['Date']=pd.to_datetime(dfNepse['Contract No'], format='%Y%m%d%H%M%f', errors='ignore')
    return FloorSheetdf

In [6]:

startDate= '2001-1-1'
count = 0
for symbol in list(df['Symbol No']):
    print("Stock No: %s\nTime: %s"%(symbol,time.ctime()))
    filename = "./data/NEPSE%s.csv"%symbol
    count +=1
    if os.path.isfile(filename):
        print("Stock No: %s Data Already Downloaded"%symbol)
    else:
        dftest=CompanyStocksTransactions(symbol,startDate)
        dftest.to_csv(filename)
    print("%s of %s completed \n"%(count,len(df['Symbol No'])))

NameError: name 'df' is not defined

In [4]:
# plot
# Need to convert Contract Number to Date-time
Stock = 'EBL'
symbol = "%d"%df[df['Stock Symbol']==Stock]['Symbol No']
print(symbol)
dfNepse=pd.read_csv("./data/NEPSE%s.csv"%symbol)
dfNepse.sort_values('Contract No',inplace = True)
startfrom=200101016341249
dfNepse=dfNepse[dfNepse['Contract No']>startfrom]
dfNepse.plot(x='Contract No', y = 'Rate')

NameError: name 'df' is not defined

In [26]:
dfNepse.describe()

,Unnamed: 0,S.N.,Contract No,Buyer Broker,Seller Broker,Quantity,Rate,Amount,Date
count,113712.000000,113712.000000,1.137120e+05,113712.000000,113712.000000,113712.000000,113712.000000,1.137120e+05,8.060100e+04
mean,73196.684624,73197.684624,2.015279e+14,30.079992,30.507668,121.117006,1907.599814,2.427043e+05,2.014676e+14
std,32914.417569,32914.417569,2.372424e+11,16.802288,17.420393,323.670104,941.953099,6.466173e+05,1.946399e+11
min,0.000000,1.000000,2.007113e+14,1.000000,1.000000,0.000000,18.000000,0.000000e+00,2.010010e+14
25%,44813.750000,44814.750000,2.014012e+14,16.000000,14.000000,40.000000,1107.000000,5.340000e+04,2.013062e+14
50%,73241.500000,73242.500000,2.016091e+14,32.000000,32.000000,87.000000,1720.000000,1.216800e+05,2.015102e+14
75%,101669.250000,101670.250000,2.017063e+14,44.000000,45.000000,120.000000,2730.000000,2.755000e+05,2.016112e+14
max,130097.000000,130098.000000,2.018073e+14,59.000000,59.000000,37500.000000,3920.000000,7.968000e+07,2.017062e+14


In [19]:
symbol = "NMB"
url = "http://nepalstock.com/marketdepth/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
tables = soup.findAll("table")
for table in tables:
    for row in table.findAll("tr")[4:]:
        col = row.findAll("td")
        print (col[0].string,": ",col[1].string)

In [15]:
soup.findAll("table")

[<table border="1" width="100%">
 <tr>
 <td><div>NMB Bank Limited (NMB)</div></td>
 </tr>
 </table>, <table border="1" width="100%">
 <tr>
 <td><table border="1" class="depthIndex" width="100%">
 <tr>
 <td><label class="livePrice">
                         316.00</label><br/>
 <span class="indicator greenIndicator"></span>
 <span class="green">1.00  
                         0.32%</span>
 </td>
 <td><label>Pr. Close </label><br/>315.00</td>
 <td><label>Open </label><br/>315.00</td>
 <td><label>High </label><br/>316.00</td>
 <td><label>Low </label><br/>313.00</td>
 <td><label>Close</label><br/>
 
                         ---</td>
 </tr>
 </table></td>
 </tr>
 <tr>
 <td><label class="livePrice">Order Book</label></td>
 </tr>
 <tr>
 <td><table border="1" class="" width="100%">
 <tr>
 <td colspan="2"><table border="1" class="table table-striped table-bordered orderTable" width="100%">
 <thead>
 <tr>
 <th>Buy Orders</th>
 <th>Buy Qty</th>
 <th>Buy Price</th>
 </tr>
 </thead>
 <tr>
 <td>3</t